In [1]:
import tkinter as tk
import pandas as pd

# 각 사전 파일의 경로
stopwords_file = 'stopwords.xlsx'
oneword_file = 'oneword.xlsx'
category_file = 'category.xlsx'
modify_file = 'modify.xlsx'

# 각 사전 파일을 읽어오기
df_stopwords = pd.read_excel(stopwords_file)
df_oneword = pd.read_excel(oneword_file)
df_category = pd.read_excel(category_file)
df_modify = pd.read_excel(modify_file)

def check_word(entry_widget, df, column_name):
    # 단어 존재 여부 확인
    word = entry_widget.get()
    if word in df[column_name].values:
        result_label.config(text="이미 등록된 단어입니다")
    else:
        result_label.config(text="등록되지 않은 단어입니다")

def add_word(entry_widget, df, file_path, column_name):
    # 사전에 단어 추가
    word = entry_widget.get()
    if word and word not in df[column_name].values:
        new_row = pd.DataFrame({column_name: [word]})
        updated_df = pd.concat([df, new_row], ignore_index=True)
        updated_df.to_excel(file_path, index=False)
        result_label.config(text="단어가 등록되었습니다")
    else:
        result_label.config(text="이미 등록된 단어입니다")

def check_and_add_substitute(before_entry, after_entry, df, file_path):
    # 치환어 확인 및 추가
    before_word = before_entry.get()
    after_word = after_entry.get()
    if before_word and before_word in df['Before'].values:
        result_label.config(text="이미 등록된 치환어입니다")
    elif before_word and after_word:
        new_row = pd.DataFrame({'Before': [before_word], 'After': [after_word]})
        updated_df = pd.concat([df, new_row], ignore_index=True)
        updated_df.to_excel(file_path, index=False)
        result_label.config(text="치환어가 등록되었습니다")
    else:
        result_label.config(text="치환 전/후 단어를 입력해주세요")

def check_and_add_category_word(category_entry, word_entry, df, file_path):
    # 카테고리와 단어 확인 및 추가
    category = category_entry.get()
    word = word_entry.get()
    if category and word:
        # 중복 확인
        if not df[(df['Category'] == category) & (df['Keyword'] == word)].empty:
            result_label.config(text="이미 존재하는 카테고리/단어 조합입니다")
        else:
            new_row = pd.DataFrame({'Category': [category], 'Keyword': [word]})
            updated_df = pd.concat([df, new_row], ignore_index=True)
            updated_df.to_excel(file_path, index=False)
            result_label.config(text="카테고리/단어가 추가되었습니다")
    else:
        result_label.config(text="카테고리와 단어를 입력해주세요")


# 현재 활성화된 인터페이스 요소들을 추적하기 위한 리스트
current_interface_elements = []

def clear_current_interface():
    # 현재 활성화된 인터페이스 요소들을 숨김
    global current_interface_elements
    for element in current_interface_elements:
        element.pack_forget()
    current_interface_elements = []


# 각 사전 관리 인터페이스 함수
def create_stopword_interface():
    clear_current_interface()
    stopword_entry = tk.Entry(window)
    stopword_entry.pack(pady=5)
    stopword_check_button = tk.Button(window, text="찾기", command=lambda: check_word(stopword_entry, df_stopwords, 'Keyword'))
    stopword_check_button.pack(pady=5)
    stopword_add_button = tk.Button(window, text="추가", command=lambda: add_word(stopword_entry, df_stopwords, stopwords_file, 'Keyword'))
    stopword_add_button.pack(pady=5)
    current_interface_elements.extend([stopword_entry, stopword_check_button, stopword_add_button])

def create_single_char_interface():
    clear_current_interface()
    single_char_entry = tk.Entry(window)
    single_char_entry.pack(pady=5)
    single_char_check_button = tk.Button(window, text="찾기", command=lambda: check_word(single_char_entry, df_oneword, 'Keyword'))
    single_char_check_button.pack(pady=5)
    single_char_add_button = tk.Button(window, text="추가", command=lambda: add_word(single_char_entry, df_oneword, oneword_file, 'Keyword'))
    single_char_add_button.pack(pady=5)
    current_interface_elements.extend([single_char_entry, single_char_check_button, single_char_add_button])

def create_substitute_interface():
    clear_current_interface()

    # "치환 전" 레이블 및 입력 필드
    substitute_before_label = tk.Label(window, text="치환 전")
    substitute_before_label.pack(pady=2)
    substitute_before_entry = tk.Entry(window)
    substitute_before_entry.pack(pady=2)

    # "치환 후" 레이블 및 입력 필드
    substitute_after_label = tk.Label(window, text="치환 후")
    substitute_after_label.pack(pady=2)
    substitute_after_entry = tk.Entry(window)
    substitute_after_entry.pack(pady=2)

    # "찾기 및 추가" 버튼
    substitute_check_button = tk.Button(window, text="찾기 및 추가", command=lambda: check_and_add_substitute(substitute_before_entry, substitute_after_entry, df_modify, modify_file))
    substitute_check_button.pack(pady=5)

    current_interface_elements.extend([substitute_before_label, substitute_before_entry, substitute_after_label, substitute_after_entry, substitute_check_button])


def create_category_interface():
    clear_current_interface()
    # 카테고리 레이블 및 입력 필드
    category_label = tk.Label(window, text="카테고리")
    category_label.pack(pady=2)
    category_entry = tk.Entry(window)
    category_entry.pack(pady=2)

    # 단어 레이블 및 입력 필드
    word_label = tk.Label(window, text="추가할 단어")
    word_label.pack(pady=2)
    word_entry = tk.Entry(window)
    word_entry.pack(pady=2)

    # 찾기 및 추가 버튼
    category_check_add_button = tk.Button(window, text="찾기 및 추가", command=lambda: check_and_add_category_word(category_entry, word_entry, df_category, category_file))
    category_check_add_button.pack(pady=5)
    current_interface_elements.extend([category_label, category_entry, word_label, word_entry, category_check_add_button])


# 메인 윈도우 생성 및 설정
window = tk.Tk()
window.title("큐피트 단어 종합 사전")
window.geometry("350x400")

# 메인 윈도우 상단에 제목 레이블 추가
title_label = tk.Label(window, text="큐피트 단어 종합 사전", font=("Helvetica", 16))
title_label.pack(pady=10)

# 결과 레이블
result_label = tk.Label(window, text="")
result_label.pack(pady=5)

# 각 사전 관리 버튼
stopword_button = tk.Button(window, text="불용어 등록", command=create_stopword_interface)
stopword_button.pack(pady=5)

single_char_button = tk.Button(window, text="한 글자 등록", command=create_single_char_interface)
single_char_button.pack(pady=5)

substitute_button = tk.Button(window, text="치환어 등록", command=create_substitute_interface)
substitute_button.pack(pady=5)

category_button = tk.Button(window, text="카테고리 사전 등록", command=create_category_interface)
category_button.pack(pady=5)

# 이벤트 루프 실행
window.mainloop()